In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/age-prediction/20-50/20-50'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import tensorflow_hub as hub
model = tf.keras.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/50-classification/versions/2")
])
model.build([None, 224, 224, 3])  # Batch input shape.

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image
import os
import pandas as pd
import numpy as np

# Define the path to the testing data directory
test_data_directory = '/kaggle/input/age-prediction/20-50/20-50/test/20'

# List all files in the directory
test_data_files = os.listdir(test_data_directory)

# Load the data into a list of NumPy arrays
image_data = []
for file in test_data_files:
    file_path = os.path.join(test_data_directory, file)
    img = Image.open(file_path)
    img_array = np.array(img)
    image_data.append(img_array)

# Convert the list of NumPy arrays to a NumPy array
test_data = np.array(image_data)

# Display the first few images
test_data[:5]  # Displaying the first 5 images as an example


In [ ]:

# Define the path to the training data directory
train_data_directory = '/kaggle/input/age-prediction/20-50/20-50/train/22'

# List all files in the directory
train_data_files = os.listdir(train_data_directory)

# Load the data into a list of NumPy arrays
image_data_train = []
for file in train_data_files:
    file_path = os.path.join(train_data_directory, file)
    img = Image.open(file_path)
    img_array = np.array(img)
    image_data_train.append(img_array)

# Convert the list of NumPy arrays to a NumPy array
train_data = np.array(image_data_train)

# Display the first few images
train_data[:5]  # Displaying the first 5 images as an example



In [ ]:
def apply_noise(model, noise):
    for layer in model.layers:
        if layer.trainable:
            weights = layer.get_weights()
            for i in range(len(weights)):
                weights[i] += noise * np.random.randn(*weights[i].shape)
            layer.set_weights(weights)

In [ ]:
# Function to perform unlearning using Noise Minimization-Maximization
def unlearn(model, forget_classes, epochs=10, learning_rate=0.001, noise_factor=0.1):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    for epoch in range(epochs):
        for class_label in forget_classes:
            noise = noise_factor * np.random.randn()  # Generate noise
            apply_noise(model, noise)

            # Perform unlearning step (using a placeholder loss, replace it with your actual loss function)
            with tf.GradientTape() as tape:
                predictions = model(np.random.randn(1, 64, 64, 3))  # Example input
                loss = tf.reduce_mean(predictions)  # Replace this with your actual loss function

            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
# List of classes to forget (replace it with your actual forget classes)
forget_classes = [0, 1, 2]

# Perform unlearning
unlearn(model, forget_classes, epochs=10, learning_rate=0.001, noise_factor=0.1)

In [ ]:

# Evaluate the original model on the test set (no accuracy calculation without labels)
predictions_original = model.predict(test_data)

# Unlearn the model
unlearn(model, forget_classes, epochs=1500, learning_rate=0.001, noise_factor=0.35)

# Evaluate the unlearned model on the test set (no accuracy calculation without labels)
predictions_unlearned = model.predict(test_data)

# Visual inspection or other evaluation methods
# Example: Print some predictions for visual inspection
print("Original Model Predictions:")
print(predictions_original[:5])

print("\nUnlearned Model Predictions:")
print(predictions_unlearned[:5])
